<a href="https://colab.research.google.com/github/marina554/accounting-practice/blob/main/Simple%20Journal%20Entry%20Manager(%E7%B0%A1%E6%98%93%E4%BB%95%E8%A8%B3%E5%B8%B3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
from datetime import datetime

# === Soft date parser ===
def parse_date_soft(date_str):
    patterns = [
        "%Y-%m-%d",
        "%Y/%m/%d",
        "%Y.%m.%d",
        "%Y %m %d"
    ]

    for p in patterns:
        try:
            return datetime.strptime(date_str, p).date()
        except ValueError:
            continue

    print(f"Warning: Invalid date format: {date_str}")
    return None


# === JournalEntry class ===
class JournalEntry:
    def __init__(self, date, category, payment_method, amount):
        self.date = parse_date_soft(date)
        self.category = category
        self.payment_method = payment_method
        self.amount = amount

        # Automatically assign debit and credit accounts
        self.debit_account, self.credit_account = self.auto_map_accounts(category, payment_method)

    def auto_map_accounts(self, category, payment_method):
        debit = category
        credit = "Cash"

        if payment_method == "Bank Transfer":
            credit = "Bank Deposit"
        elif payment_method == "Credit Card":
            credit = "Accounts Payable"

        if category == "Sales":
            debit, credit = "Bank Deposit", "Sales Revenue"

        return debit, credit

    def to_dict(self):
        return {
            "Date": self.date,
            "Debit": self.debit_account,
            "Credit": self.credit_account,
            "Amount": self.amount,
            "Category": self.category,
            "Payment Method": self.payment_method,
        }

    def info(self):
        return f"{self.date}: {self.category}, {self.amount} JPY [{self.debit_account} / {self.credit_account}]"


# === Journal class ===
class Journal:
    def __init__(self):
        self.entries = []

    def add_entry(self, entry):
        if not isinstance(entry, JournalEntry):
            raise ValueError("Please provide a JournalEntry object")
        self.entries.append(entry)

    def add_raw_entry(self, date, category, payment_method, amount):
        try:
            entry = JournalEntry(date, category, payment_method, amount)
            self.add_entry(entry)
        except ValueError as e:
            print(f"Warning: {e}")

    def to_dataframe(self):
        return pd.DataFrame([e.to_dict() for e in self.entries])

    def total_amount(self):
        return sum(e.amount for e in self.entries)

    def total_by_category(self):
        df = self.to_dataframe()
        return df.groupby("Category")["Amount"].sum()

    def show_all_info(self):
        for e in self.entries:
            print(e.info())


# === Sample data ===
journal_raw = [
    ("2025-11-01", "Office Supplies", "Cash", 5000),
    ("2025-11-02", "Travel Expense", "Credit Card", 12000),
    ("2025-11-03", "Sales", "Bank Transfer", 30000),
    ("2025-11-04", "Communication Expense", "Bank Transfer", 8000),
    ("2025/11/05", "Unknown Expense", "Cash", -1000),  # Invalid format
]

journal_obj = Journal()

for rec in journal_raw:
    journal_obj.add_raw_entry(*rec)

# === Output ===
print("=== All Journal Entries ===")
journal_obj.show_all_info()

print("\n=== DataFrame ===")
df = journal_obj.to_dataframe()
print(df)

print("\n=== Total Amount ===")
print(journal_obj.total_amount())

print("\n=== Total by Category ===")
print(journal_obj.total_by_category())


=== All Journal Entries ===
2025-11-01: Office Supplies, 5000 JPY [Office Supplies / Cash]
2025-11-02: Travel Expense, 12000 JPY [Travel Expense / Accounts Payable]
2025-11-03: Sales, 30000 JPY [Bank Deposit / Sales Revenue]
2025-11-04: Communication Expense, 8000 JPY [Communication Expense / Bank Deposit]
2025-11-05: Unknown Expense, -1000 JPY [Unknown Expense / Cash]

=== DataFrame ===
         Date                  Debit            Credit  Amount  \
0  2025-11-01        Office Supplies              Cash    5000   
1  2025-11-02         Travel Expense  Accounts Payable   12000   
2  2025-11-03           Bank Deposit     Sales Revenue   30000   
3  2025-11-04  Communication Expense      Bank Deposit    8000   
4  2025-11-05        Unknown Expense              Cash   -1000   

                Category Payment Method  
0        Office Supplies           Cash  
1         Travel Expense    Credit Card  
2                  Sales  Bank Transfer  
3  Communication Expense  Bank Transfer  
4 

In [12]:
import pandas as pd
from datetime import datetime

# === 柔らかい日付パーサー ===
def parse_date_soft(date_str):
    patterns = [
        "%Y-%m-%d",
        "%Y/%m/%d",
        "%Y.%m.%d",
        "%Y %m %d"
    ]

    for p in patterns:
        try:
            return datetime.strptime(date_str, p).date()
        except ValueError:
            continue

    # 全部ダメだったときだけ警告
    print(f"警告: 日付が不正です: {date_str}")
    return None


# === JournalEntry クラス ===
class JournalEntry:
    def __init__(self, date, category, payment_method, amount):
        self.date = parse_date_soft(date)
        self.category = category
        self.payment_method = payment_method
        self.amount = amount

        # 借方・貸方を自動設定
        self.debit_account, self.credit_account = self.auto_map_accounts(category, payment_method)

    def auto_map_accounts(self, category, payment_method):
        # 簡易ルール
        debit = category
        credit = "現金"

        if payment_method == "銀行振込":
            credit = "普通預金"
        elif payment_method == "クレジットカード":
            credit = "未払金"

        if category == "売上":
            debit, credit = "普通預金", "売上高"

        return debit, credit

    def to_dict(self):
        return {
            "日付": self.date,
            "借方科目": self.debit_account,
            "貸方科目": self.credit_account,
            "金額": self.amount,
            "区分": self.category,
            "支払方法": self.payment_method,
        }

    def info(self):
        return f"{self.date}: {self.category}, {self.amount}円 [{self.debit_account} / {self.credit_account}]"


# === Journal クラス ===
class Journal:
    def __init__(self):
        self.entries = []

    def add_entry(self, entry):
        if not isinstance(entry, JournalEntry):
            raise ValueError("JournalEntryオブジェクトを渡してください")
        self.entries.append(entry)

    def add_raw_entry(self, date, category, payment_method, amount):
        try:
            entry = JournalEntry(date, category, payment_method, amount)
            self.add_entry(entry)
        except ValueError as e:
            print(f"警告: {e}")

    def to_dataframe(self):
        return pd.DataFrame([e.to_dict() for e in self.entries])

    def total_amount(self):
        return sum(e.amount for e in self.entries)

    def total_by_category(self):
        df = self.to_dataframe()
        return df.groupby("区分")["金額"].sum()

    def show_all_info(self):
        for e in self.entries:
            print(e.info())


# === サンプルデータ ===
journal_raw = [
    ("2025-11-01", "事務用品費", "現金", 5000),
    ("2025-11-02", "旅費交通費", "クレジットカード", 12000),
    ("2025-11-03", "売上", "銀行振込", 30000),
    ("2025-11-04", "通信費", "銀行振込", 8000),
    ("2025/11/05", "不明費用", "現金", -1000),  # 不正データ
]

journal_obj = Journal()

for rec in journal_raw:
    journal_obj.add_raw_entry(*rec)

# === 出力 ===
print("【全仕訳一覧】")
journal_obj.show_all_info()

print("\n【DataFrame表示】")
df = journal_obj.to_dataframe()
print(df)

print("\n【合計金額】")
print(journal_obj.total_amount())

print("\n【科目ごとの合計】")
print(journal_obj.total_by_category())


【全仕訳一覧】
2025-11-01: 事務用品費, 5000円 [事務用品費 / 現金]
2025-11-02: 旅費交通費, 12000円 [旅費交通費 / 未払金]
2025-11-03: 売上, 30000円 [普通預金 / 売上高]
2025-11-04: 通信費, 8000円 [通信費 / 普通預金]
2025-11-05: 不明費用, -1000円 [不明費用 / 現金]

【DataFrame表示】
           日付   借方科目  貸方科目     金額     区分      支払方法
0  2025-11-01  事務用品費    現金   5000  事務用品費        現金
1  2025-11-02  旅費交通費   未払金  12000  旅費交通費  クレジットカード
2  2025-11-03   普通預金   売上高  30000     売上      銀行振込
3  2025-11-04    通信費  普通預金   8000    通信費      銀行振込
4  2025-11-05   不明費用    現金  -1000   不明費用        現金

【合計金額】
54000

【科目ごとの合計】
区分
不明費用     -1000
事務用品費     5000
売上       30000
旅費交通費    12000
通信費       8000
Name: 金額, dtype: int64
